In [1]:
import os
os.chdir('..')
os.getcwd()

'/Users/signapoop/Desktop/fyp-graph-clustering'

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt

import pickle 
import numpy as np
import pandas as pd
import torch
import os
from timeit import default_timer as timer
import networkx as nx
import scipy.sparse as sp

In [3]:
from core.SimpleNet import SimpleNet
from core.GraphConvNet import GraphConvNet
from core.EmbeddingDataSet import EmbeddingDataSet
from core.GraphDataBlock import GraphDataBlock
from util.plot_graph_embedding import plot_graph_embedding
from util.evaluation_metrics import evaluate_viz_metrics, trustworthiness, neighborhood_preservation
from util.network_utils import get_net_projection, _get_net_projection
from util.graph_utils import get_shortest_path_matrix, neighbor_sampling

cuda not available
cuda not available


In [4]:
from sklearn.manifold import TSNE

In [5]:
if torch.cuda.is_available():
    dtypeFloat = torch.cuda.FloatTensor
    dtypeLong = torch.cuda.LongTensor
    print('cuda available')
    device = 'cuda'
else:
    dtypeFloat = torch.FloatTensor
    dtypeLong = torch.LongTensor
    print('cuda not available')
    device = 'cpu'

cuda not available


In [6]:
dataset_name = 'reddit_full'
parent_dir = os.path.abspath('..')
input_dir = os.path.join(parent_dir, 'data')
dataset = EmbeddingDataSet(dataset_name, input_dir, train=True)
dataset.create_all_data(n_batches=1, shuffle=False)
dataset.summarise()

Data blocks of length:  [23699]
Time to create all data (s) = 0.0975
Name of dataset = reddit_full
Input dimension = 602
Number of training samples = 23699
Training labels = True


In [8]:
X = dataset.inputs

In [9]:
np.mean(X, axis=0)

array([ 2.29590437e-13,  1.57702133e-13,  8.10965474e-17,  2.44042705e-17,
       -4.16070121e-17,  5.72936731e-18,  1.80032368e-17, -8.10450159e-18,
       -3.31675557e-18,  2.00668396e-17, -9.54035693e-18,  2.54284593e-17,
        9.76428478e-17, -2.06969295e-17,  7.80889810e-17, -5.57266466e-17,
        4.28859305e-17, -8.59077169e-17,  7.00500649e-17,  5.01190811e-17,
       -3.98104362e-17,  2.75623324e-17,  5.15080896e-17,  4.83318745e-17,
       -1.93899029e-17,  3.61423294e-18,  4.46309750e-17, -6.24936712e-18,
        2.98648541e-18,  6.24468244e-18,  2.96891785e-17, -5.49934938e-17,
       -1.34965717e-17,  2.98086379e-17,  3.02677369e-17, -3.56691764e-17,
        2.92089985e-17, -2.01207135e-17,  7.96396110e-19,  6.32432205e-19,
       -2.64142923e-17,  3.27927810e-17, -1.02828792e-18,  3.76367432e-17,
       -4.21293542e-17,  1.67278318e-17,  1.06119782e-17, -2.67917021e-17,
        3.28864747e-17,  2.04252179e-17,  2.61405312e-18,  1.02594558e-18,
        1.40821571e-17,  

In [10]:
sampled_block = dataset.get_sample_block(n_initial=50, sample_neighbors=True, verbose=1)

Initial set of 50 points was expanded to 3060 points


In [11]:
X_sampled = sampled_block.inputs.numpy()
X_sampled.shape

(3060, 602)

In [12]:
embedder = TSNE(n_components=2, perplexity=30, metric='cosine', verbose=0)
y_pred_tsne_cos = embedder.fit_transform(X_sampled)

In [13]:
embedder = TSNE(n_components=2, perplexity=30, metric='euclidean', verbose=0)
y_pred_tsne_euc = embedder.fit_transform(X_sampled)

In [14]:
res = evaluate_viz_metrics(y_pred_tsne_cos, sampled_block, verbose=1)

One NN accuracy = 0.5078
Avg graph distance = 2.0880
Avg feature distance = 0.5050
Total distance = 2.5930


In [15]:
res = evaluate_viz_metrics(y_pred_tsne_euc, sampled_block, verbose=1)

One NN accuracy = 0.4624
Avg graph distance = 1.8456
Avg feature distance = 0.8565
Total distance = 2.7022


In [ ]:
feats = X_sampled
feats[:,0] = np.log(X_sampled[:,0]+1.0)
feats[:,1] = np.log(X_sampled[:,1]-min(np.min(X_sampled[:,1]), -1))

In [ ]:
embedder = TSNE(n_components=2, perplexity=30, metric='cosine', verbose=0)
y_pred_tsne_cos_second = embedder.fit_transform(feats)

In [ ]:
res = evaluate_viz_metrics(y_pred_tsne_cos_second, sampled_block, verbose=1)

In [ ]:
embedder = TSNE(n_components=2, perplexity=30, metric='euclidean', verbose=0)
y_pred_tsne_euc_second = embedder.fit_transform(feats)

In [ ]:
res = evaluate_viz_metrics(y_pred_tsne_euc_second, sampled_block, verbose=1)

In [ ]:
np.mean(feats, axis=0)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
Z = scaler.fit_transform(X_sampled)

In [ ]:
np.mean(Z, axis=0)

In [ ]:
embedder = TSNE(n_components=2, perplexity=30, metric='cosine', verbose=0)
y_pred_tsne_cos_third = embedder.fit_transform(Z)

In [ ]:
res = evaluate_viz_metrics(y_pred_tsne_cos_third, sampled_block, verbose=1)

In [ ]:
embedder = TSNE(n_components=2, perplexity=30, metric='euclidean', verbose=0)
y_pred_tsne_euc_third = embedder.fit_transform(Z)

In [ ]:
res = evaluate_viz_metrics(y_pred_tsne_euc_third, sampled_block, verbose=1)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
Z = scaler.fit_transform(X)

In [ ]:
Z.shape

In [ ]:
import pickle
with open('reddit_full.pkl', 'wb') as f:
    pickle.dump([Z, dataset.labels, dataset.adj_matrix], f)

In [ ]:
dataset_name = 'reddit_full'
parent_dir = os.path.abspath('..')
input_dir = os.path.join(parent_dir, 'data')
dataset = EmbeddingDataSet(dataset_name, input_dir, train=True)
dataset.create_all_data(n_batches=1, shuffle=False)
dataset.summarise()

In [ ]:
np.mean(dataset.inputs.toarray(), axis=0)